## PyCity Schools Analysis

##### Top and Bottom Performing Schools
- The most immediate observation is that Charter schools populate the top 5 performing schools and District schools populate the bottom 5 performing schools for standardized test scores. 
- Overall, District schools had lower standardized test scores than Charter schools.

##### Subject Grades
- Math and Reading standardized test scores seem to stay the same across the board between grades in a school. 
- If the test grades deviated wildly from each other between different grades, further analysis would be required. That doesn't appear to be the case.
- Therefore, any solutions that would improve one grade's performance would probably work for all the other grades as well, across most of these schools.

##### Budget & Population
- Large Schools (population greater than 2000 students) appear to have a significantly lower overall passing rate than schools with less than 2000 students, by about 20%.
- It appears the more that schools spent on their students in \$\$ amounts, the less well they performed on these standardized tests.
- Charter schools have smaller budgets than District schools. They also have fewer students.

##### Conclusions
- Given the observations and conditions above, one of the conclusions we might draw from this dataset is that a higher school budget doesn't necessarily mean better performance on standardized tests.
- Perhaps there is a correlation between the *size* of a school and performance on standardized tests, however, and it would be a good set of data to delve further into.
- Specifically, this data shows that the more students a school has, the worse its performance on standardized tests.

##### Opinions
- This could be the result of numerous factors; in my opinion it would be due to the lack of attention to every individual student in the time crunch required of teachers to get their lessons across. However this data doesn't necessarily prove that theory in and of itself so we would require further data. Perhaps something like *Average Amount of Time Teacher Spends With Individual Students* or something similar.

##### Other 
- Why is there someone named 'Dr. Richard Scott' in the student list under Huang High School? Are some teachers actually mixed in with these students? What if some of them don't have prefixes like 'Dr.' in their name? Does this data need further scrubbing? My numbers seem generally the same as the example results we were shown so I'm just going to trust the teachers on this one.

In [1]:
import pandas as pd

In [2]:
#import schools and then students file
schools_file = "Resources/schools_complete.csv"
students_file = "Resources/students_complete.csv"

In [3]:
#read one dataframe for schools
schools_df = pd.read_csv(schools_file)
# schools_df.head()

In [4]:
#rename 'name' column to 'school' to prepare for merge
schools_df = schools_df.rename(columns={"name":"school"})
# schools_df.head()

In [5]:
#read second dataframe for students
students_df = pd.read_csv(students_file)
# students_df.head()

In [6]:
#outer merge on 'school' column
merged_df = pd.merge(schools_df, students_df, on="school", how="outer")
merged_df.head()

,School ID,school,type,size,budget,Student ID,name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [7]:
#check for missing data
#merged_df.count()
#check columns
merged_df.columns

Index(['School ID', 'school', 'type', 'size', 'budget', 'Student ID', 'name',
       'gender', 'grade', 'reading_score', 'math_score'],
      dtype='object')

## District Summary

In [8]:
#Create Dataframe with headers: 'Total Schools', 'Total Students,' 'Total Budget', 'Avg Math Score', 'Avg Reading Score',
#'% Passing Math', '% Passing Reading,' % Overall Passing Rate'
totalschools = merged_df['school'].nunique()
totalschools

15

In [9]:
totalstudents = len(merged_df['name'])
totalstudents

39170

In [10]:
totalbudget = sum(schools_df['budget'])
totalbudget

24649428

In [11]:
#score averages
mathscore = merged_df['math_score'].mean()
readingscore = merged_df['reading_score'].mean()

In [12]:
#passing score formulas
#return the amount of math scores above 65
#revision: actually this assignment considers 'above 70' to be passing
#anyway this formula passes the amount of students whose scores are over 70. all little harvard prodigies.
passmath = len(merged_df.loc[merged_df['math_score'] > 70, ['math_score']])
passmath

28356

In [13]:
#average is the amount of passing scores divided by all students. example file reads 72.392137
#if passing score is '65' i get 0.83 which doesn't match the example file
#if passing scores is '70' i get '0.72392137' which matches the example file. so i'll go with that then.
#would be nice if the readme file specified what counts as a passing score.
percmath = passmath / totalstudents * 100
percmath

72.39213683941792

In [14]:
passreading = len(merged_df.loc[merged_df['reading_score'] > 70, ['reading_score']])
percreading = passreading / totalstudents * 100
#according to the example this should read 82.971662
percreading

82.97166198621395

In [15]:
#get overall passing rate "(Average of the above two)" reading and writing and put it all in a new DF
#in the example the result is clearly wrong and gives 80.431606 as the average of 72.392137 and 82.971662
percpassing = (percmath + percreading) / 2
percpassing

77.68189941281594

In [16]:
#putting it together
dist_summary = pd.DataFrame({"Total Schools": totalschools,
                            "Total Students": totalstudents,
                            "Total Budget": totalbudget,
                            "Average Math Score": mathscore,
                            "Average Reading Score": readingscore,
                            "% Passing Math": percmath,
                            "% Passing Reading": percreading,
                            "% Overall Passing Rate": percpassing}, index=[0])

#formating
dist_summary["Total Budget"] = dist_summary["Total Budget"].map("${:,.2f}".format)

dist_summary = dist_summary[["Total Schools",
                                            "Total Students",
                                            "Total Budget",
                                            "Average Math Score",
                                            "% Passing Math",
                                            "% Passing Reading",
                                            "% Overall Passing Rate"]]

dist_summary

,Total Schools,Total Students,Total Budget,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,72.392137,82.971662,77.681899


## School Summary

In [17]:
#Group by school, show 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score',
#'% Passing Math', '% Passing Reading', '% Overall Passing Rate'
school_group = merged_df.groupby(['school'])

In [18]:
#basic stuff
#i borrowed this schooltype thing from someone else's solution because it took me like 6 hours to figure this out to no avail.
#takeaway: strings are annoying
schooltype = schools_df.set_index('school')["type"]
totalstudents = school_group['name'].count()
totschoolbud = school_group['budget'].mean()
perstudentbud = totschoolbud / totalstudents


In [19]:
#average math score
mathscore = school_group['math_score'].mean()

#average reading score
readingscore = school_group['reading_score'].mean()

#% passing math
#also took these % passing from another solution because i couldn't figure out why i can't use .loc with groupby
passmath = merged_df[merged_df['math_score'] > 70].groupby('school')['Student ID'].count() / totalstudents * 100

#% passing reading
passreading = merged_df[merged_df['reading_score'] > 70].groupby('school')['Student ID'].count() / totalstudents * 100

#% overall passing rate
percpassing = (passmath + passreading) / 2

In [20]:
school_summary = pd.DataFrame({"School Type": schooltype,
                              "Total Students": totalstudents,
                              "Total School Budget": totschoolbud,
                              "Per Student Budget": perstudentbud,
                              "Average Math Score": mathscore,
                              "Average Reading Score": readingscore,
                              "% Passing Math": passmath,
                              "% Passing Reading": passreading,
                              "% Overall Passing": percpassing})


#formating
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)

school_summary = school_summary[["School Type",
                                "Total Students",
                                "Total School Budget",
                                "Per Student Budget",
                                "Average Math Score",
                                "Average Reading Score",
                                "% Passing Math",
                                "% Passing Reading",
                                "% Overall Passing"]]

school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",628.0,77.048432,81.033963,64.630225,79.300643,71.965434
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,89.558665,93.864370,91.711518
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,63.750424,78.433367,71.091896
Ford High School,District,2739,"$1,763,916.00",644.0,77.102592,80.746258,65.753925,77.510040,71.631982
Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.816757,89.713896,93.392371,91.553134
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,64.746494,78.187702,71.467098
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,90.632319,92.740047,91.686183
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4761,"$3,094,650.00",650.0,77.072464,80.966394,63.852132,78.281874,71.067003
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.044699,91.683992,92.203742,91.943867


## Top Performing Schools (By Passing Rate)

In [21]:
#sort by overall passing and put highest on top
top_schools = school_summary.sort_values(['% Overall Passing'], ascending=False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Wilson High School,Charter,2283,"$1,319,574.00",578.0,83.274201,83.989488,90.932983,93.254490,92.093736
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.044699,91.683992,92.203742,91.943867
Wright High School,Charter,1800,"$1,049,400.00",583.0,83.682222,83.955000,90.277778,93.444444,91.861111
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,89.558665,93.864370,91.711518
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,90.632319,92.740047,91.686183


## Bottom Performing Schools (By Passing Rate)

In [22]:
#"I put my thing down, flip it and reverse it" - Missy Elliot, 2002
bottom_schools = school_summary.sort_values(['% Overall Passing'], ascending=True)
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",637.0,76.842711,80.744686,64.066017,77.744436,70.905226
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4761,"$3,094,650.00",650.0,77.072464,80.966394,63.852132,78.281874,71.067003
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,63.750424,78.433367,71.091896
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,64.746494,78.187702,71.467098


## Math Scores by Grade

In [23]:
# breaking down this formula:
# first it locates all entries matching '9th' under the 'grade' column in students_df
# then it groups them by 'school'
# then it returns the mean of the values in the math_score column
# the index is still the schools from the previous dataframes so we don't have to do anything

#yes, this would probably work a lot better in a loop of some sort but i'm against the clock here
ninth = students_df.loc[students_df['grade'] == '9th'].groupby('school')['math_score'].mean()
tenth = students_df.loc[students_df['grade'] == '10th'].groupby('school')['math_score'].mean()
eleventh = students_df.loc[students_df['grade'] == '11th'].groupby('school')['math_score'].mean()
twelfth = students_df.loc[students_df['grade'] == '12th'].groupby('school')['math_score'].mean()

math_grade = pd.DataFrame({"9th": ninth.map("{:,.2f}".format),
                          "10th": tenth.map("{:,.2f}".format),
                          "11th": eleventh.map("{:,.2f}".format),
                          "12th": twelfth.map("{:,.2f}".format)})

math_grade = math_grade [['9th', '10th', '11th', '12th']]

math_grade

,9th,10th,11th,12th
school,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Scores by Grade

In [24]:
ninth = students_df.loc[students_df['grade'] == '9th'].groupby('school')['reading_score'].mean()
tenth = students_df.loc[students_df['grade'] == '10th'].groupby('school')['reading_score'].mean()
eleventh = students_df.loc[students_df['grade'] == '11th'].groupby('school')['reading_score'].mean()
twelfth = students_df.loc[students_df['grade'] == '12th'].groupby('school')['reading_score'].mean()

reading_grade = pd.DataFrame({"9th": ninth.map("{:,.2f}".format),
                          "10th": tenth.map("{:,.2f}".format),
                          "11th": eleventh.map("{:,.2f}".format),
                          "12th": twelfth.map("{:,.2f}".format)})

reading_grade = reading_grade [['9th', '10th', '11th', '12th']]

reading_grade

,9th,10th,11th,12th
school,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [25]:
#creating bins based on example, then making a new variable to hold them
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
merged_df["Spending Range"] = pd.cut(merged_df['budget']/merged_df['size'], spending_bins, labels=group_names)
# school_summary

In [26]:
spending_group = merged_df.groupby("Spending Range")

#Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, % Overall Passing Rate
avgmath = spending_group['math_score'].mean()
avgreading = spending_group['reading_score'].mean()

#passing math
passmath = merged_df[merged_df['math_score'] > 70].groupby("Spending Range")['Student ID'].count() / spending_group['Student ID'].count() * 100

#% passing reading
passreading = merged_df[merged_df['reading_score'] > 70].groupby("Spending Range")['Student ID'].count() / spending_group['Student ID'].count() * 100
#% overall passing rate
percpassing = (passmath + passreading) / 2

In [27]:
spending_scores = pd.DataFrame({"Average Math Score": avgmath.map("{:,.2f}".format),
                               "Average Reading Score": avgreading.map("{:,.2f}".format),
                               "% Passing Math": passmath.map("{:,.2f}".format),
                               "% Passing Reading": passreading.map("{:,.2f}".format),
                               "% Overall Passing Rate": percpassing.map("{:,.2f}".format)})

spending_scores = spending_scores[['Average Math Score', 
                                   'Average Reading Score', 
                                   '% Passing Math', 
                                   '% Passing Reading', 
                                   '% Overall Passing Rate']]

spending_scores.index.name = "Spending Ranges (Per Student)"

spending_scores

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.36,83.96,90.33,93.45,91.89
$585-615,83.53,83.84,90.53,92.47,91.50
$615-645,78.06,81.43,68.96,80.95,74.95
$645-675,77.05,81.01,64.06,78.37,71.22


## Scores by School Size

In [28]:
#just using the Example's bins
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
merged_df['School Sizes'] = pd.cut(merged_df['size'], size_bins, labels = group_names)

In [29]:
size_group = merged_df.groupby('School Sizes')

#Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, % Overall Passing Rate
avgmath = size_group['math_score'].mean()
avgreading = size_group['reading_score'].mean()

#passing math
passmath = merged_df[merged_df['math_score'] > 70].groupby("School Sizes")['Student ID'].count() / size_group['Student ID'].count() * 100

#% passing reading
passreading = merged_df[merged_df['reading_score'] > 70].groupby("School Sizes")['Student ID'].count() / size_group['Student ID'].count() * 100
#% overall passing rate
percpassing = (passmath + passreading) / 2

In [30]:
size_scores = pd.DataFrame({"Average Math Score": avgmath.map("{:,.2f}".format),
                               "Average Reading Score": avgreading.map("{:,.2f}".format),
                               "% Passing Math": passmath.map("{:,.2f}".format),
                               "% Passing Reading": passreading.map("{:,.2f}".format),
                               "% Overall Passing Rate": percpassing.map("{:,.2f}".format)})

size_scores = size_scores[['Average Math Score', 
                                   'Average Reading Score', 
                                   '% Passing Math', 
                                   '% Passing Reading', 
                                   '% Overall Passing Rate']]

size_scores.index.name = "School Size"

size_scores

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.83,83.97,91.36,92.37,91.86
Medium (1000-2000),83.37,83.87,89.93,93.25,91.59
Large (2000-5000),77.48,81.20,66.38,79.53,72.96


## Scores by School Type

In [31]:
#group by school type
type_group = merged_df.groupby('type')

#Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, % Overall Passing Rate
avgmath = type_group['math_score'].mean()
avgreading = type_group['reading_score'].mean()

#passing math
passmath = merged_df[merged_df['math_score'] > 70].groupby("type")['Student ID'].count() / type_group['Student ID'].count() * 100

#% passing reading
passreading = merged_df[merged_df['reading_score'] > 70].groupby("type")['Student ID'].count() / type_group['Student ID'].count() * 100
#% overall passing rate
percpassing = (passmath + passreading) / 2

In [32]:
type_scores = pd.DataFrame({"Average Math Score": avgmath.map("{:,.2f}".format),
                               "Average Reading Score": avgreading.map("{:,.2f}".format),
                               "% Passing Math": passmath.map("{:,.2f}".format),
                               "% Passing Reading": passreading.map("{:,.2f}".format),
                               "% Overall Passing Rate": percpassing.map("{:,.2f}".format)})

type_scores = type_scores[['Average Math Score', 
                                   'Average Reading Score', 
                                   '% Passing Math', 
                                   '% Passing Reading', 
                                   '% Overall Passing Rate']]

type_scores.index.name = "School Type"

type_scores

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.41,83.90,90.28,93.15,91.72
District,76.99,80.96,64.31,78.37,71.34
